In [1]:
pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 15.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=39b19308c1ae809b8a76fbb33c770b2ee4a532edd7e137ed4de9d89c6287825f
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [27]:
import whisper
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Transcribe audios
total_recordings = 36
model = whisper.load_model("base")

speech_to_text_outputs = []
for i in range(1, total_recordings+1):
    filename = f"audio{i}.m4a"
    result = model.transcribe(filename)
    text = result["text"]
    speech_to_text_outputs.append(text)

#  Dictionaries (BoW)
acads_lite_BoW = {
    "Events": ["oasis", "bosm", "fest", "prof show", "concert","apogee"],
    "Campus/Places": ["dcc","pcc", "anc", "club", "clubs", "department","pizzeria","stalls","stall","looters","cvr","mela","mele"],
    "People/Slang": ["party", "pandeyji", "farre","far", "looters", "redi","reddy" ,"lite","light"],
    "Games": ["pubg", "valorant", "valo", "bgmi", "cod", "call of duty",
              "clash of clans", "coc", "clash royale", "gaming", "game"],
    "Cards/Gambling": ["casino", "cards", "patte", "taash", "chits"],
    "Substance/Slang": ["nashe", "nasha", "maal", "redbull", "monster"],
    "Clubs":["cris","treat"]
}

ghot_BoW = {
    "Coding/Technical": ["cp", "code", "coding", "debug", "debugging", "tech"],
    "Exams/Tests": ["lab test", "tut test", "compre", "midsem", "exam", "quiz","study","midsem","medicine"],
    "Academics/Subjects": ["gen chem", "maths", "m1", "m2", "m3","mechsol",
                           "workshop","bio","pns","techre", "meow"],
    "Study Material": ["slides","slide", "notes", "studydeck"],
    "Places": ["library", "s9","nab"],
    "Slang/Motivation": ["phoda", "padhai", "padha", "padhle"],
    "Grades": ["a", "a-", "b", "b-", "c", "c-","nc","av"],
    "Lectures": ["lecture","nalanda"]
}

#  Label each transcript
found_labels = []

for text in speech_to_text_outputs:
    lowered_text = text.lower()

    total_ghot_words = sum(
        (w in lowered_text)
        for words in ghot_BoW.values()
        for w in words
    )

    total_acads_lite_words = sum(
        (w in lowered_text)
        for words in acads_lite_BoW.values()
        for w in words
    )

    if total_acads_lite_words < total_ghot_words:
        found_labels.append("GHOT")
    else:
        found_labels.append("ACADS LITE")

# Train TF-IDF + Logistic Regression
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(speech_to_text_outputs)
clf = LogisticRegression()
clf.fit(X, found_labels)

print("Training complete!!!")

# Function for new audio
def classify_new_audio(filename):
    result = model.transcribe(filename)
    text = result["text"]

    X_new = vectorizer.transform([text])
    pred = clf.predict(X_new)[0]
    return text, pred




Training complete!!!


In [34]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, clf.predict(X_test)))
print(confusion_matrix(y_test, clf.predict(X_test)))


              precision    recall  f1-score   support

  ACADS LITE       0.00      0.00      0.00         3
        GHOT       0.62      1.00      0.77         5

    accuracy                           0.62         8
   macro avg       0.31      0.50      0.38         8
weighted avg       0.39      0.62      0.48         8

[[0 3]
 [0 5]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, found_labels, test_size=0.2, random_state=42)

# Retrain the model on the training data
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate F1-score and Recall
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"F1-score: {f1:.2f}")
print(f"Recall: {recall:.2f}")

F1-score: 0.48
Recall: 0.62


In [32]:
# Test on new audio

test_file = "test11.m4a"

transcript, predicted_label = classify_new_audio(test_file)

# print("Transcript:", transcript)
print("Predicted Label:", predicted_label)

Transcript:  Our schedule for the night would be to grab a burger from DCC and then play Valorant all night along with sipping monster.
Predicted Label: ACADS LITE


In [25]:
# @title
from collections import Counter

# Count occurrences of GHOT and ACADS LITE
label_counts = Counter(found_labels)

print("Number of GHOT:", label_counts["GHOT"])
print("Number of ACADS LITE:", label_counts["ACADS LITE"])


Number of GHOT: 4
Number of ACADS LITE: 32
